## The purpose of this notebook is to turn the datasets returned from the lag creator to be grouped by the day or hour and add the tweet text for that day or hour

In [1]:
#imports
import pandas as pd
import os
import gc
import sys

In [2]:
#get data with following lag
granularity = "days"
lag = 1
# analyser = "TextBlob"
analyser = "vader"

#how to group - by "day" or "hour"
group_by = "day"

#if to use cleaned data
cleaned = True
#if with polarity
with_polarity = True

In [3]:
#prepare filename to read
folder = "/root/fyp/datasets/tweets_and_prices/vader" if with_polarity else  "/root/fyp/datasets/tweets_and_prices/cleaned";

#if cleaned and with polarity add the cleaned dir
if(cleaned and with_polarity):
    folder = folder+"/cleaned"

print(folder)    
    
filename = folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"

/root/fyp/datasets/tweets_and_prices/vader/cleaned


## Getting tweet volumes

In [27]:
#load all tweets if cleaned

#prepare original filename to read
# original_folder = "/root/fyp/datasets/tweets_and_prices/vader"
# original_filename = original_folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"
original_filename = '/root/fyp/datasets/general/with_sentiment/vader/cleaned/tweets.csv'

In [28]:
original_filename

'/root/fyp/datasets/general/with_sentiment/vader/cleaned/tweets.csv'

In [29]:
tweets_original = pd.read_csv(original_filename) if cleaned else pd.read_csv(filename)
from datetime import datetime
from datetime import timedelta

tweets_original['timestamp'] = tweets_original['date'].str[:10]
tweets_original['timestamp'] = pd.to_datetime(tweets_original['timestamp'], utc = True)


#choose lag
lag = 1
#this can be hours or days
granularity = "days"
#move by lag 
tweets_original['timestamp'] = (tweets_original['timestamp'] - timedelta(hours=lag)) if granularity == 'hours' else (tweets_original['timestamp'] - timedelta(days=lag))

/tmp/ipykernel_875/1601572849.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets_original = pd.read_csv(original_filename) if cleaned else pd.read_csv(filename)


In [30]:
if(cleaned):
    tweets_original["dt"] = pd.to_datetime(tweets_original['timestamp'])
    tweets_original = tweets_original.drop(['timestamp'], axis=1)
    
#floor datetimes to day or hour
tweets_original['DateTime'] = tweets_original['dt'].dt.floor(group_by[0])

In [31]:
tweets_original.head()

Unnamed: 0.4  Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0             0             1             1             1     18683.0   
1             1             2             2             2     18684.0   
2             2             3             3             3     18685.0   
3             3             4             4             4     18686.0   
4             4             5             5             5     18687.0   

     index                user_name      user_location  \
0  18683.0          Bitcoin Reserve                NaN   
1  18684.0        BitcoinGratis.com  BitcoinGratis.com   
2  18685.0             CryptoSquawk          Australia   
3  18686.0             CryptoSquawk          Australia   
4  18687.0  solomon gift_cryptoMind                NaN   

                                    user_description         user_created  \
0  Premier bitcoin brokerage.\n\nRobust. Forward-...  2020-01-16 18:34:25   
1  #Bitcoin $BTC #Cryptocurrency #Crypto #Ethereu...  2018-10-28 19:18:18   
2  24x7 Crypto market real-time audio squawk for ...  2017-10-05 10:13:09   
3  24x7 Crypto market real-time audio squawk for ...  2017-10-05 10:13:09   
4  #Bitcoin \nCrypto, tech and blockchain enthusi...  2020-09-01 21:05:48   

   ...                                           hashtags  \
0  ...                                        ['Bitcoin']   
1  ...  ['EGLD', 'Elrond', 'elrondnetwork', 'bitcoin',...   
2  ...                       ['Bitcoin', 'crypto', 'BTC']   
3  ...                       ['Bitcoin', 'crypto', 'BTC']   
4  ...             ['Bitcoin', 'BTC', 'Ethereum', 'defi']   

                source  is_retweet    year compound pos_pol neg_pol neu_pol  \
0            TweetDeck       False  2021.0    0.000   0.000     0.0   1.000   
1      Twitter Web App       False  2021.0    0.000   0.000     0.0   1.000   
2      CryptoSquawkBot       False  2021.0    0.000   0.000     0.0   1.000   
3      CryptoSquawkBot       False  2021.0    0.296   0.216     0.0   0.784   
4  Twitter for Android       False  2021.0    0.340   0.130     0.0   0.870   

                         dt                  DateTime  
0 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  
1 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  
2 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  
3 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  
4 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  

[5 rows x 26 columns]

In [47]:
#group tweet volumes daily
grouped_volumes = tweets_original.groupby(pd.Grouper(key='dt',freq=group_by[0])).size().reset_index(name='tweet_vol')

In [48]:
#set index to datetime
grouped_volumes.index = pd.to_datetime(grouped_volumes.index)
grouped_volumes = grouped_volumes.set_index('dt')

In [49]:
#set type of volumes to float
grouped_volumes['tweet_vol'] = grouped_volumes['tweet_vol'].astype(float)

In [50]:
grouped_volumes.head()

tweet_vol
dt                                  
2021-02-04 00:00:00+00:00     1575.0
2021-02-05 00:00:00+00:00     3054.0
2021-02-06 00:00:00+00:00     2871.0
2021-02-07 00:00:00+00:00     5277.0
2021-02-08 00:00:00+00:00     4067.0

In [14]:
grouped_volumes.to_csv('/root/grouped_volumes.csv')

In [23]:
tweets_original.head()

Unnamed: 0.4  Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0             0             1             1             1     18683.0   
1             1             2             2             2     18684.0   
2             2             3             3             3     18685.0   
3             3             4             4             4     18686.0   
4             4             5             5             5     18687.0   

     index                user_name      user_location  \
0  18683.0          Bitcoin Reserve                NaN   
1  18684.0        BitcoinGratis.com  BitcoinGratis.com   
2  18685.0             CryptoSquawk          Australia   
3  18686.0             CryptoSquawk          Australia   
4  18687.0  solomon gift_cryptoMind                NaN   

                                    user_description         user_created  \
0  Premier bitcoin brokerage.\n\nRobust. Forward-...  2020-01-16 18:34:25   
1  #Bitcoin $BTC #Cryptocurrency #Crypto #Ethereu...  2018-10-28 19:18:18   
2  24x7 Crypto market real-time audio squawk for ...  2017-10-05 10:13:09   
3  24x7 Crypto market real-time audio squawk for ...  2017-10-05 10:13:09   
4  #Bitcoin \nCrypto, tech and blockchain enthusi...  2020-09-01 21:05:48   

   ...                                           hashtags  \
0  ...                                        ['Bitcoin']   
1  ...  ['EGLD', 'Elrond', 'elrondnetwork', 'bitcoin',...   
2  ...                       ['Bitcoin', 'crypto', 'BTC']   
3  ...                       ['Bitcoin', 'crypto', 'BTC']   
4  ...             ['Bitcoin', 'BTC', 'Ethereum', 'defi']   

                source  is_retweet    year compound pos_pol neg_pol neu_pol  \
0            TweetDeck       False  2021.0    0.000   0.000     0.0   1.000   
1      Twitter Web App       False  2021.0    0.000   0.000     0.0   1.000   
2      CryptoSquawkBot       False  2021.0    0.000   0.000     0.0   1.000   
3      CryptoSquawkBot       False  2021.0    0.296   0.216     0.0   0.784   
4  Twitter for Android       False  2021.0    0.340   0.130     0.0   0.870   

                         dt                  DateTime  
0 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  
1 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  
2 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  
3 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  
4 2021-02-05 00:00:00+00:00 2021-02-05 00:00:00+00:00  

[5 rows x 26 columns]

In [11]:
close_prices = pd.Series(tweets_original.loc[tweets_original.groupby('DateTime').dt.idxmax()].set_index('DateTime')['Close'])

#get final prices of day - uncomment if needed
# tweets_df["Close"] = pd.Series(tweets_original.loc[tweets_original.groupby('DateTime').dt.idxmax()]["Close"]).values
# df_copy["Open"] = pd.Series(df.loc[df.groupby('DateTime').dt.idxmin()]["Open"]).values
# df_copy["High"] = pd.Series(df.loc[df.groupby('DateTime').Close.idxmax()]["High"]).values
# df_copy["Low"] = pd.Series(df.loc[df.groupby('DateTime').Close.idxmin()]["Low"]).values

In [12]:
#to free memory space
del tweets_original

## Group tweets

In [4]:
filename

'/root/fyp/datasets/tweets_and_prices/vader/cleaned/final_data_lag_days_1.csv'

In [5]:
#read data
df = pd.read_csv(filename)

In [6]:
df.head()

Unnamed: 0.1  Unnamed: 0        Date  priceUSD  transactions      size  \
0             0        3856  2021/02/06   39803.0        333727  894488.0   
1             1        3856  2021/02/06   39803.0        333727  894488.0   
2             2        3856  2021/02/06   39803.0        333727  894488.0   
3             3        3856  2021/02/06   39803.0        333727  894488.0   
4             4        3856  2021/02/06   39803.0        333727  894488.0   

   sentbyaddress    difficulty      hashrate  mining_profitability  ...  \
0         548661  2.138445e+13  1.700209e+20                  0.26  ...   
1         548661  2.138445e+13  1.700209e+20                  0.26  ...   
2         548661  2.138445e+13  1.700209e+20                  0.26  ...   
3         548661  2.138445e+13  1.700209e+20                  0.26  ...   
4         548661  2.138445e+13  1.700209e+20                  0.26  ...   

   price7rocUSD  price14rocUSD  price30rocUSD  price90rocUSD  \
0        16.868         23.054          4.665        161.984   
1        16.868         23.054          4.665        161.984   
2        16.868         23.054          4.665        161.984   
3        16.868         23.054          4.665        161.984   
4        16.868         23.054          4.665        161.984   

                   timestamp  \
0  2021-02-05 00:00:00+00:00   
1  2021-02-05 00:00:00+00:00   
2  2021-02-05 00:00:00+00:00   
3  2021-02-05 00:00:00+00:00   
4  2021-02-05 00:00:00+00:00   

                                                text  compound  pos_pol  \
0  Grayscale Bitcoin Trust grew by 157 b 2020 86 ...    0.1779    0.208   
1  Important interview USER USER on essential inf...    0.2023    0.122   
2  If you purchase small quantity of everyday You...   -0.3182    0.099   
3  USER evelynkush evelynkush USER Ilove how supe...    0.7783    0.362   
4  Thousands of athome trader many using Reddit f...    0.0772    0.075   

   neg_pol  neu_pol  
0    0.164    0.629  
1    0.000    0.878  
2    0.212    0.690  
3    0.000    0.638  
4    0.000    0.925  

[5 rows x 745 columns]

In [7]:
#replace empty text with empty string
df['text'] = df['text'].fillna('')

In [16]:
df.columns

Index(['Unnamed: 0.1', 'Date', 'priceUSD', 'transactions', 'size',
       'sentbyaddress', 'difficulty', 'hashrate', 'mining_profitability',
       'sentinusdUSD',
       ...
       'price14rocUSD', 'price30rocUSD', 'price90rocUSD', 'text', 'compound',
       'pos_pol', 'neg_pol', 'neu_pol', 'dt', 'DateTime'],
      dtype='object', length=745)

In [18]:
#remove extra columns
df = df.drop(['Unnamed: 0.1','Date'], axis=1)

In [9]:
#get datetime
df["dt"] = pd.to_datetime(df['timestamp'])
df = df.drop(['timestamp'], axis=1)

In [10]:
#set DateTime 
df['DateTime'] = df['dt'].dt.floor(group_by[0])

In [11]:
#set datetime to index of dataframe
df.index = pd.to_datetime(df.index)

In [19]:
df.head()

priceUSD  transactions      size  \
1970-01-01 00:00:00.000000000   39803.0        333727  894488.0   
1970-01-01 00:00:00.000000001   39803.0        333727  894488.0   
1970-01-01 00:00:00.000000002   39803.0        333727  894488.0   
1970-01-01 00:00:00.000000003   39803.0        333727  894488.0   
1970-01-01 00:00:00.000000004   39803.0        333727  894488.0   

                               sentbyaddress    difficulty      hashrate  \
1970-01-01 00:00:00.000000000         548661  2.138445e+13  1.700209e+20   
1970-01-01 00:00:00.000000001         548661  2.138445e+13  1.700209e+20   
1970-01-01 00:00:00.000000002         548661  2.138445e+13  1.700209e+20   
1970-01-01 00:00:00.000000003         548661  2.138445e+13  1.700209e+20   
1970-01-01 00:00:00.000000004         548661  2.138445e+13  1.700209e+20   

                               mining_profitability  sentinusdUSD  \
1970-01-01 00:00:00.000000000                  0.26  1.845732e+10   
1970-01-01 00:00:00.000000001                  0.26  1.845732e+10   
1970-01-01 00:00:00.000000002                  0.26  1.845732e+10   
1970-01-01 00:00:00.000000003                  0.26  1.845732e+10   
1970-01-01 00:00:00.000000004                  0.26  1.845732e+10   

                               transactionfeesUSD  median_transaction_feeUSD  \
1970-01-01 00:00:00.000000000               14.11                      7.167   
1970-01-01 00:00:00.000000001               14.11                      7.167   
1970-01-01 00:00:00.000000002               14.11                      7.167   
1970-01-01 00:00:00.000000003               14.11                      7.167   
1970-01-01 00:00:00.000000004               14.11                      7.167   

                               ...  price14rocUSD  price30rocUSD  \
1970-01-01 00:00:00.000000000  ...         23.054          4.665   
1970-01-01 00:00:00.000000001  ...         23.054          4.665   
1970-01-01 00:00:00.000000002  ...         23.054          4.665   
1970-01-01 00:00:00.000000003  ...         23.054          4.665   
1970-01-01 00:00:00.000000004  ...         23.054          4.665   

                               price90rocUSD  \
1970-01-01 00:00:00.000000000        161.984   
1970-01-01 00:00:00.000000001        161.984   
1970-01-01 00:00:00.000000002        161.984   
1970-01-01 00:00:00.000000003        161.984   
1970-01-01 00:00:00.000000004        161.984   

                                                                            text  \
1970-01-01 00:00:00.000000000  Grayscale Bitcoin Trust grew by 157 b 2020 86 ...   
1970-01-01 00:00:00.000000001  Important interview USER USER on essential inf...   
1970-01-01 00:00:00.000000002  If you purchase small quantity of everyday You...   
1970-01-01 00:00:00.000000003  USER evelynkush evelynkush USER Ilove how supe...   
1970-01-01 00:00:00.000000004  Thousands of athome trader many using Reddit f...   

                               compound  pos_pol  neg_pol  neu_pol  \
1970-01-01 00:00:00.000000000    0.1779    0.208    0.164    0.629   
1970-01-01 00:00:00.000000001    0.2023    0.122    0.000    0.878   
1970-01-01 00:00:00.000000002   -0.3182    0.099    0.212    0.690   
1970-01-01 00:00:00.000000003    0.7783    0.362    0.000    0.638   
1970-01-01 00:00:00.000000004    0.0772    0.075    0.000    0.925   

                                                     dt  \
1970-01-01 00:00:00.000000000 2021-02-05 00:00:00+00:00   
1970-01-01 00:00:00.000000001 2021-02-05 00:00:00+00:00   
1970-01-01 00:00:00.000000002 2021-02-05 00:00:00+00:00   
1970-01-01 00:00:00.000000003 2021-02-05 00:00:00+00:00   
1970-01-01 00:00:00.000000004 2021-02-05 00:00:00+00:00   

                                               DateTime  
1970-01-01 00:00:00.000000000 2021-02-05 00:00:00+00:00  
1970-01-01 00:00:00.000000001 2021-02-05 00:00:00+00:00  
1970-01-01 00:00:00.000000002 2021-02-05 00:00:00+00:00  
1970-01-01 00:00:00.000000003 2021-02-05 00:00:00+00:00  
1970-0

In [22]:
df = df.drop(['text'], axis=1)

In [ ]:
df = df.drop(['dt'], axis=1)

In [23]:
#group by day and take the average of values in that day
#df['text'] = df['text'].astype(str)
# df_copy['text'] = df_copy.groupby(['DateTime'])['text'].transform(lambda x : ''.join(x + " "))
#tweets_grouped = df.groupby('DateTime').agg(lambda x : ''.join(x + ' '))
#group by day and take the average of values in that day
df_grouped = df.groupby('DateTime').agg(lambda x: x.mean())
#df_grouped['text'] = tweets_grouped

In [37]:
len(df_grouped)

142

In [25]:
df_grouped.head()

priceUSD  transactions      size  sentbyaddress  \
DateTime                                                                     
2021-02-05 00:00:00+00:00   39803.0      333727.0  894488.0       548661.0   
2021-02-06 00:00:00+00:00   38705.0      286208.0  849970.0       482676.0   
2021-02-07 00:00:00+00:00   40920.0      335061.0  852911.0       567828.0   
2021-02-08 00:00:00+00:00   46682.0      359994.0  905196.0       622948.0   
2021-02-09 00:00:00+00:00   45766.0      340615.0  901470.0       597506.0   

                             difficulty      hashrate  mining_profitability  \
DateTime                                                                      
2021-02-05 00:00:00+00:00  2.138445e+13  1.700209e+20                 0.260   
2021-02-06 00:00:00+00:00  2.143440e+13  1.695037e+20                 0.242   
2021-02-07 00:00:00+00:00  2.143440e+13  1.744655e+20                 0.269   
2021-02-08 00:00:00+00:00  2.143440e+13  1.587651e+20                 0.321   
2021-02-09 00:00:00+00:00  2.143440e+13  1.600358e+20                 0.304   

                           sentinusdUSD  transactionfeesUSD  \
DateTime                                                      
2021-02-05 00:00:00+00:00  1.845732e+10              14.110   
2021-02-06 00:00:00+00:00  1.630172e+10              11.453   
2021-02-07 00:00:00+00:00  3.125926e+10              16.386   
2021-02-08 00:00:00+00:00  3.994468e+10              25.467   
2021-02-09 00:00:00+00:00  2.897420e+10              22.595   

                           median_transaction_feeUSD  ...  price3rocUSD  \
DateTime                                              ...                 
2021-02-05 00:00:00+00:00                      7.167  ...         8.900   
2021-02-06 00:00:00+00:00                      5.430  ...         3.129   
2021-02-07 00:00:00+00:00                      8.885  ...         8.807   
2021-02-08 00:00:00+00:00                     13.752  ...        17.283   
2021-02-09 00:00:00+00:00                     12.101  ...        18.245   

                           price7rocUSD  price14rocUSD  price30rocUSD  \
DateTime                                                                
2021-02-05 00:00:00+00:00        16.868         23.054          4.665   
2021-02-06 00:00:00+00:00        15.703         20.050         -2.725   
2021-02-07 00:00:00+00:00        21.419         22.676          1.306   
2021-02-08 00:00:00+00:00        34.575         45.649         17.502   
2021-02-09 00:00:00+00:00        25.216         46.815         33.370   

                           price90rocUSD  compound   pos_pol   neg_pol  \
DateTime                                                                 
2021-02-05 00:00:00+00:00        161.984  0.257043  0.199649  0.068266   
2021-02-06 00:00:00+00:00        151.620  0.290287  0.200871  0.060296   
2021-02-07 00:00:00+00:00        167.040  0.305655  0.206275  0.059643   
2021-02-08 00:00:00+00:00        199.539  0.288023  0.204604  0.053405   
2021-02-09 00:00:00+00:00        187.500  0.288799  0.198116  0.058716   

                            neu_pol                                  dt  
DateTime                                                                 
2021-02-05 00:00:00+00:00  0.732089           2021-02-05 00:00:00+00:00  
2021-02-06 00:00:00+00:00  0.738829           2021-02-06 00:00:00+00:00  
2021-02-07 00:00:00+00:00  0.734099           2021-02-07 00:00:00+00:00  
2021-02-08 00:00:00+00:00  0.741985           2021-02-08 00:00:00+00:00  
2021-02-09 00:00:00+00:00  0.743150 2021-02-08 23:59:59.999999744+00:00  

[5 rows x 741 columns]

In [51]:
#grouped_volumes['dt'] = grouped_volumes.index

In [24]:
#for memory space
del df
del tweets_grouped

In [53]:
grouped_volumes = grouped_volumes.reset_index()

In [57]:
df_grouped = pd.merge(df_grouped,grouped_volumes,on='dt',how='left')

In [55]:
#add tweet volume to df
#df_grouped['Tweet_vol'] = grouped_volumes['tweet_vol']
#df_grouped['Close'] = close_prices

In [27]:
#remove rows where close is null
#df_grouped.dropna(subset = ["Close"], inplace=True)

In [29]:
#df_grouped.head()

id   replies      likes   retweets  \
DateTime                                                                  
2018-08-24 00:00:00+00:00  1.033070e+18  1.300971  21.330097   7.854369   
2018-08-25 00:00:00+00:00  1.033351e+18  0.125874   4.755245   2.825175   
2018-08-26 00:00:00+00:00  1.033700e+18  0.842857  19.300000   4.228571   
2018-08-27 00:00:00+00:00  1.034094e+18  6.764331  48.738854  18.745223   
2018-08-28 00:00:00+00:00  1.034441e+18  2.536842  24.189474   8.205263   

                             Close  Volume_(BTC)  Volume_(Currency)  \
DateTime                                                              
2018-08-24 00:00:00+00:00  7027.99      5.263317       36749.800785   
2018-08-25 00:00:00+00:00  7187.23      3.664420       26193.723043   
2018-08-26 00:00:00+00:00  7275.00      2.817791       20387.920017   
2018-08-27 00:00:00+00:00  7301.01      2.142963       15556.419160   
2018-08-28 00:00:00+00:00  7350.00      3.419014       25051.550529   

                           Weighted_Price   neg_pol   neu_pol   pos_pol  \
DateTime                                                                  
2018-08-24 00:00:00+00:00     6979.691936  0.031903  0.852311  0.115796   
2018-08-25 00:00:00+00:00     7096.869104  0.039671  0.853007  0.107336   
2018-08-26 00:00:00+00:00     7233.424594  0.039157  0.820357  0.140486   
2018-08-27 00:00:00+00:00     7260.409426  0.037892  0.841885  0.120229   
2018-08-28 00:00:00+00:00     7317.004481  0.037947  0.839863  0.122184   

                           compound                                  dt  \
DateTime                                                                  
2018-08-24 00:00:00+00:00  0.267373 2018-08-24 17:29:23.615384576+00:00   
2018-08-25 00:00:00+00:00  0.201412 2018-08-25 12:34:15.953488384+00:00   
2018-08-26 00:00:00+00:00  0.232191 2018-08-26 12:07:15.114155264+00:00   
2018-08-27 00:00:00+00:00  0.228047 2018-08-27 12:22:26.258503168+00:00   
2018-08-28 00:00:00+00:00  0.226857 2018-08-28 12:10:40.315865088+00:00   

                                                                        text  \
DateTime                                                                       
2018-08-24 00:00:00+00:00      LATEST PRICES FOR TOP 10 CRYPTOCURRENCIES ...   
2018-08-25 00:00:00+00:00  Total Market Cap 217213 253599 1 BTC 6738 58 B...   
2018-08-26 00:00:00+00:00  Market Cap 216843 015604 BTC Dominance 5368 BT...   
2018-08-27 00:00:00+00:00  Total Market Cap 215621 588022 1 BTC 6709 33 B...   
2018-08-28 00:00:00+00:00  Total Market Cap 225058 112278 1 BTC 6919 17 B...   

                           Tweet_vol  
DateTime                              
2018-08-24 00:00:00+00:00      506.0  
2018-08-25 00:00:00+00:00      857.0  
2018-08-26 00:00:00+00:00      796.0  
2018-08-27 00:00:00+00:00      885.0  
2018-08-28 00:00:00+00:00     1012.0

In [62]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Columns: 742 entries, priceUSD to tweet_vol
dtypes: datetime64[ns, UTC](1), float64(741)
memory usage: 823.3 KB


In [ ]:
#save file
filename = folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"
df_grouped.to_csv(filename)

In [59]:
filename

'/root/fyp/datasets/tweets_and_prices/vader/cleaned/final_data_lag_days_1.csv'

In [64]:
df_grouped.head()

priceUSD  transactions      size  sentbyaddress    difficulty  \
0   39803.0      333727.0  894488.0       548661.0  2.138445e+13   
1   38705.0      286208.0  849970.0       482676.0  2.143440e+13   
2   40920.0      335061.0  852911.0       567828.0  2.143440e+13   
3   46682.0      359994.0  905196.0       622948.0  2.143440e+13   
4   45766.0      340615.0  901470.0       597506.0  2.143440e+13   

       hashrate  mining_profitability  sentinusdUSD  transactionfeesUSD  \
0  1.700209e+20                 0.260  1.845732e+10              14.110   
1  1.695037e+20                 0.242  1.630172e+10              11.453   
2  1.744655e+20                 0.269  3.125926e+10              16.386   
3  1.587651e+20                 0.321  3.994468e+10              25.467   
4  1.600358e+20                 0.304  2.897420e+10              22.595   

   median_transaction_feeUSD  ...  price7rocUSD  price14rocUSD  price30rocUSD  \
0                      7.167  ...        16.868         23.054          4.665   
1                      5.430  ...        15.703         20.050         -2.725   
2                      8.885  ...        21.419         22.676          1.306   
3                     13.752  ...        34.575         45.649         17.502   
4                     12.101  ...        25.216         46.815         33.370   

   price90rocUSD  compound   pos_pol   neg_pol   neu_pol  \
0        161.984  0.257043  0.199649  0.068266  0.732089   
1        151.620  0.290287  0.200871  0.060296  0.738829   
2        167.040  0.305655  0.206275  0.059643  0.734099   
3        199.539  0.288023  0.204604  0.053405  0.741985   
4        187.500  0.288799  0.198116  0.058716  0.743150   

                                   dt  tweet_vol  
0           2021-02-05 00:00:00+00:00     3054.0  
1           2021-02-06 00:00:00+00:00     2871.0  
2           2021-02-07 00:00:00+00:00     5277.0  
3           2021-02-08 00:00:00+00:00     4067.0  
4 2021-02-08 23:59:59.999999744+00:00        NaN  

[5 rows x 742 columns]

In [61]:
df_grouped.shape

(142, 742)

In [68]:
df_grouped['dt'].describe()

count                                    142
mean     2021-09-22 11:29:34.647887360+00:00
min                2021-02-05 00:00:00+00:00
25%                2021-06-03 12:00:00+00:00
50%                2021-08-19 12:00:00+00:00
75%                2022-01-19 18:00:00+00:00
max                2022-07-17 00:00:00+00:00
Name: dt, dtype: object

In [67]:
df_grouped['compound'].describe()

count    142.000000
mean       0.351928
std        0.074034
min        0.148924
25%        0.295505
50%        0.356822
75%        0.407652
max        0.555886
Name: compound, dtype: float64